In [4]:
# Importer les bibliothèques nécessaires
import pandas as pd
import requests
import json

# Charger de la clé API à partir du fichier de configuration
def load_api_key():
    try:
        with open('local.config.json', 'r') as config_file:
            config_data = json.load(config_file)
            return config_data.get('api_key', '')
    except FileNotFoundError:
        print("Le fichier de configuration 'local.config.json' n'a pas été trouvé.")
        return ''


# Définir la fonction pour obtenir le code commune à partir d'un SIRET
def get_commune_from_siret(siret, api_key):
    url = f'https://api.insee.fr/entreprises/sirene/V3/siret/{siret}'
    
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Accept': 'application/json'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if 'etablissement' in data and data['etablissement']:
            code_commune = data['etablissement']['adresseEtablissement']['codeCommuneEtablissement']
            return code_commune
    else:
        print(f"Erreur: {response.status_code}")
        return None


# Charger la clé API à partir du fichier de configuration
cle_api = load_api_key()

# Charger le fichier CSV avec pandas
df = pd.read_csv('enrichsiret.csv')

# Appliquer la fonction pour obtenir le département et créer une nouvelle colonne
df['Commune'] = df['SIRET'].apply(lambda x: get_commune_from_siret(str(x), cle_api))

# Enregistrer le DataFrame résultant dans un nouveau fichier CSV
df.to_csv('resultat_pandas.csv', index=False)